# Suffix Trie para Análisis de Secuencias de ADN
## Conversión de SARS-CoV-2 a Aminoácidos

Este notebook implementa un suffix trie simple para analizar secuencias de ADN del SARS-CoV-2 y convertir codones a aminoácidos.

In [1]:
# Implementación simple del Suffix Trie para ADN
class Nodo:
    """Nodo del suffix trie - versión simple"""
    def __init__(self, caracter):
        self.caracter = caracter          # Nucleótido (A, T, G, C)
        self.posiciones = []              # Posiciones donde aparece este sufijo
        self.es_final = False             # ¿Es el final de un sufijo?
        self.hijos = {}                   # Diccionario de hijos {caracter: nodo}

class SuffixTrie:
    """Suffix Trie simple para secuencias de ADN"""
    def __init__(self):
        self.raiz = Nodo('$')  # Nodo raíz especial
    
    def construir_trie(self, secuencia_adn):
        """
        Construye el suffix trie insertando todos los sufijos de la secuencia
        """
        print(f"Construyendo suffix trie para secuencia de {len(secuencia_adn)} nucleótidos...")
        
        # Insertar todos los sufijos
        for i in range(len(secuencia_adn)):
            sufijo = secuencia_adn[i:]
            self._insertar_sufijo(sufijo, i)
        
        print("✅ Suffix trie construido exitosamente!")
        return self
    
    def _insertar_sufijo(self, sufijo, posicion_inicial):
        """Inserta un sufijo en el trie"""
        nodo_actual = self.raiz
        
        # Recorrer cada nucleótido del sufijo
        for i, nucleotido in enumerate(sufijo):
            # Si no existe el hijo para este nucleótido, crearlo
            if nucleotido not in nodo_actual.hijos:
                nodo_actual.hijos[nucleotido] = Nodo(nucleotido)
            
            # Moverse al hijo y agregar la posición
            nodo_actual = nodo_actual.hijos[nucleotido]
            nodo_actual.posiciones.append(posicion_inicial + i)
        
        # Marcar el último nodo como final del sufijo
        nodo_actual.es_final = True
    
    def buscar_patron(self, patron):
        """
        Busca un patrón en el trie y retorna las posiciones donde aparece
        """
        nodo_actual = self.raiz
        
        # Navegar siguiendo el patrón
        for nucleotido in patron:
            if nucleotido not in nodo_actual.hijos:
                return []  # Patrón no encontrado
            nodo_actual = nodo_actual.hijos[nucleotido]
        
        # Retornar las posiciones del último nodo del patrón
        return sorted(list(set(nodo_actual.posiciones)))
    
    def estadisticas_trie(self):
        """Muestra estadísticas del trie construido"""
        def contar_nodos(nodo):
            count = 1  # Contar este nodo
            for hijo in nodo.hijos.values():
                count += contar_nodos(hijo)
            return count
        
        total_nodos = contar_nodos(self.raiz)
        print(f"📊 Estadísticas del Suffix Trie:")
        print(f"   Total de nodos: {total_nodos}")
        print(f"   Nucleótidos en raíz: {list(self.raiz.hijos.keys())}")

In [2]:
# Tabla de conversión de codones a aminoácidos
TABLA_GENETICA = {
    # Fenilalanina
    'TTT': 'F', 'TTC': 'F',
    # Leucina
    'TTA': 'L', 'TTG': 'L', 'CTT': 'L', 'CTC': 'L', 'CTA': 'L', 'CTG': 'L',
    # Serina
    'TCT': 'S', 'TCC': 'S', 'TCA': 'S', 'TCG': 'S', 'AGT': 'S', 'AGC': 'S',
    # Tirosina
    'TAT': 'Y', 'TAC': 'Y',
    # Cisteína
    'TGT': 'C', 'TGC': 'C',
    # Triptófano
    'TGG': 'W',
    # Prolina
    'CCT': 'P', 'CCC': 'P', 'CCA': 'P', 'CCG': 'P',
    # Histidina
    'CAT': 'H', 'CAC': 'H',
    # Glutamina
    'CAA': 'Q', 'CAG': 'Q',
    # Arginina
    'CGT': 'R', 'CGC': 'R', 'CGA': 'R', 'CGG': 'R', 'AGA': 'R', 'AGG': 'R',
    # Isoleucina
    'ATT': 'I', 'ATC': 'I', 'ATA': 'I',
    # Metionina (START)
    'ATG': 'M',
    # Treonina
    'ACT': 'T', 'ACC': 'T', 'ACA': 'T', 'ACG': 'T',
    # Asparagina
    'AAT': 'N', 'AAC': 'N',
    # Lisina
    'AAA': 'K', 'AAG': 'K',
    # Valina
    'GTT': 'V', 'GTC': 'V', 'GTA': 'V', 'GTG': 'V',
    # Alanina
    'GCT': 'A', 'GCC': 'A', 'GCA': 'A', 'GCG': 'A',
    # Ácido aspártico
    'GAT': 'D', 'GAC': 'D',
    # Ácido glutámico
    'GAA': 'E', 'GAG': 'E',
    # Glicina
    'GGT': 'G', 'GGC': 'G', 'GGA': 'G', 'GGG': 'G',
    # Codones de parada (STOP)
    'TAA': '*', 'TAG': '*', 'TGA': '*'
}

def adn_a_aminoacidos(secuencia_adn, frame=0):
    """
    Convierte secuencia de ADN a cadena de aminoácidos
    frame: marco de lectura (0, 1, o 2)
    """
    print(f"🧬 Convirtiendo ADN a aminoácidos (marco de lectura {frame})...")
    
    # Ajustar por el marco de lectura
    secuencia_ajustada = secuencia_adn[frame:]
    aminoacidos = []
    
    # Procesar de 3 en 3 nucleótidos (codones)
    for i in range(0, len(secuencia_ajustada) - 2, 3):
        codon = secuencia_ajustada[i:i+3]
        
        # Convertir a aminoácido si existe en la tabla
        if codon in TABLA_GENETICA:
            aminoacido = TABLA_GENETICA[codon]
            aminoacidos.append(aminoacido)
            
            # Si encontramos un codón de parada, podemos terminar
            if aminoacido == '*':
                print(f"⚠️  Codón de parada encontrado en posición {i//3}")
                break
        else:
            # Codón no reconocido
            aminoacidos.append('X')
    
    secuencia_proteina = ''.join(aminoacidos)
    print(f"✅ Conversión completada: {len(aminoacidos)} aminoácidos")
    return secuencia_proteina

def encontrar_orfs(secuencia_adn):
    """
    Encuentra Open Reading Frames (ORFs) en la secuencia
    Un ORF empieza con ATG (metionina) y termina con codón de parada
    """
    print("🔍 Buscando Open Reading Frames (ORFs)...")
    orfs = []
    
    # Buscar en los 3 marcos de lectura
    for frame in range(3):
        secuencia_frame = secuencia_adn[frame:]
        
        i = 0
        while i < len(secuencia_frame) - 5:
            # Buscar codón de inicio ATG
            if secuencia_frame[i:i+3] == 'ATG':
                inicio = i + frame
                
                # Buscar codón de parada
                for j in range(i + 3, len(secuencia_frame) - 2, 3):
                    codon = secuencia_frame[j:j+3]
                    if codon in ['TAA', 'TAG', 'TGA']:
                        fin = j + 3 + frame
                        longitud = fin - inicio
                        
                        if longitud >= 60:  # ORF mínimo de 20 aminoácidos
                            orf_seq = secuencia_adn[inicio:fin]
                            orfs.append({
                                'inicio': inicio,
                                'fin': fin,
                                'longitud': longitud,
                                'frame': frame,
                                'secuencia': orf_seq
                            })
                        break
                
            i += 3
    
    # Ordenar por longitud (más largos primero)
    orfs.sort(key=lambda x: x['longitud'], reverse=True)
    
    print(f"✅ Se encontraron {len(orfs)} ORFs potenciales")
    return orfs

In [3]:
# Cargar y procesar la secuencia de SARS-CoV-2
def cargar_secuencia_fasta(ruta_archivo):
    """Carga secuencia desde archivo FASTA"""
    print(f"📁 Cargando secuencia desde: {ruta_archivo}")
    
    with open(ruta_archivo, 'r') as archivo:
        lineas = archivo.readlines()
    
    # La primera línea es el header (>), el resto es la secuencia
    secuencia = ''.join(linea.strip() for linea in lineas if not linea.startswith('>'))
    secuencia = secuencia.upper()  # Convertir a mayúsculas
    
    print(f"✅ Secuencia cargada: {len(secuencia)} nucleótidos")
    print(f"   Primeros 60 nucleótidos: {secuencia[:60]}")
    return secuencia

# Cargar la secuencia de SARS-CoV-2
ruta_sars = "/Users/opsystem/Downloads/Algoritmos/Situacion Problema 1 - DNA Strings/SARS-COV-2-MN908947.3.txt"
secuencia_sars = cargar_secuencia_fasta(ruta_sars)

# Análisis básico de la secuencia
print(f"\n📊 ANÁLISIS DE LA SECUENCIA SARS-CoV-2:")
print(f"   Longitud total: {len(secuencia_sars)} nucleótidos")

# Contar nucleótidos
conteo = {'A': secuencia_sars.count('A'), 
          'T': secuencia_sars.count('T'), 
          'G': secuencia_sars.count('G'), 
          'C': secuencia_sars.count('C')}

print(f"   Composición nucleótidos:")
for nucl, count in conteo.items():
    porcentaje = (count / len(secuencia_sars)) * 100
    print(f"     {nucl}: {count} ({porcentaje:.1f}%)")

# Calcular contenido GC
gc_content = (conteo['G'] + conteo['C']) / len(secuencia_sars) * 100
print(f"   Contenido GC: {gc_content:.1f}%")

📁 Cargando secuencia desde: /Users/opsystem/Downloads/Algoritmos/Situacion Problema 1 - DNA Strings/SARS-COV-2-MN908947.3.txt
✅ Secuencia cargada: 29903 nucleótidos
   Primeros 60 nucleótidos: ATTAAAGGTTTATACCTTCCCAGGTAACAAACCAACCAACTTTCGATCTCTTGTAGATCT

📊 ANÁLISIS DE LA SECUENCIA SARS-CoV-2:
   Longitud total: 29903 nucleótidos
   Composición nucleótidos:
     A: 8954 (29.9%)
     T: 9594 (32.1%)
     G: 5863 (19.6%)
     C: 5492 (18.4%)
   Contenido GC: 38.0%


In [4]:
# Construir suffix trie para una muestra de la secuencia
# (Usamos solo los primeros 1000 nucleótidos para ejemplo - el genoma completo sería muy grande)
muestra_secuencia = secuencia_sars[:1000]
print(f"🔬 Construyendo suffix trie para muestra de {len(muestra_secuencia)} nucleótidos")
print(f"   Muestra: {muestra_secuencia[:50]}...")

# Crear y construir el suffix trie
trie_sars = SuffixTrie()
trie_sars.construir_trie(muestra_secuencia)

# Mostrar estadísticas
trie_sars.estadisticas_trie()

# Buscar patrones importantes en virología
patrones_importantes = [
    'ATG',    # Codón de inicio
    'TAA',    # Codón de parada 1
    'TAG',    # Codón de parada 2
    'TGA',    # Codón de parada 3
    'AAAAAA', # Poliadenilación
    'TTTTT',  # Poliuridilación
    'CG',     # Dinucleótido CpG (importante en regulación)
]

print(f"\n🔍 BÚSQUEDA DE PATRONES IMPORTANTES:")
for patron in patrones_importantes:
    posiciones = trie_sars.buscar_patron(patron)
    if posiciones:
        print(f"   '{patron}': {len(posiciones)} apariciones en posiciones {posiciones[:10]}{'...' if len(posiciones) > 10 else ''}")
    else:
        print(f"   '{patron}': No encontrado")

🔬 Construyendo suffix trie para muestra de 1000 nucleótidos
   Muestra: ATTAAAGGTTTATACCTTCCCAGGTAACAAACCAACCAACTTTCGATCTC...
Construyendo suffix trie para secuencia de 1000 nucleótidos...
✅ Suffix trie construido exitosamente!
📊 Estadísticas del Suffix Trie:
   Total de nodos: 496264
   Nucleótidos en raíz: ['A', 'T', 'G', 'C']

🔍 BÚSQUEDA DE PATRONES IMPORTANTES:
   'ATG': 16 apariciones en posiciones [108, 267, 409, 469, 490, 508, 514, 519, 595, 727]...
   'TAA': 21 apariciones en posiciones [4, 26, 68, 79, 131, 135, 138, 142, 169, 263]...
   'TAG': 11 apariciones en posiciones [55, 113, 235, 424, 427, 532, 563, 668, 700, 761]...
   'TGA': 17 apariciones en posiciones [155, 252, 434, 458, 524, 569, 695, 719, 728, 779]...
   'AAAAAA': No encontrado
   'TTTTT': No encontrado
   'CG': 49 apariciones en posiciones [44, 71, 100, 123, 151, 164, 173, 197, 203, 207]...
✅ Suffix trie construido exitosamente!
📊 Estadísticas del Suffix Trie:
   Total de nodos: 496264
   Nucleótidos en raíz: ['

In [5]:
# Convertir la secuencia completa de SARS-CoV-2 a aminoácidos
print("=" * 60)
print("🧬 CONVERSIÓN COMPLETA DE SARS-CoV-2 A AMINOÁCIDOS")
print("=" * 60)

# Convertir en los 3 marcos de lectura
for frame in range(3):
    print(f"\n--- MARCO DE LECTURA {frame} ---")
    proteina = adn_a_aminoacidos(secuencia_sars, frame)
    
    print(f"Longitud de la proteína: {len(proteina)} aminoácidos")
    print(f"Primeros 50 aminoácidos: {proteina[:50]}")
    
    # Contar aminoácidos
    if len(proteina) > 0:
        aa_unicos = set(proteina.replace('*', '').replace('X', ''))
        print(f"Aminoácidos diferentes: {len(aa_unicos)} tipos")
        print(f"Codones de parada encontrados: {proteina.count('*')}")
        print(f"Codones no reconocidos: {proteina.count('X')}")

# Buscar ORFs (Open Reading Frames)
print(f"\n" + "=" * 60)
print("🔬 BÚSQUEDA DE OPEN READING FRAMES (ORFs)")
print("=" * 60)

orfs = encontrar_orfs(secuencia_sars)

if orfs:
    print(f"\n🎯 TOP 5 ORFs MÁS LARGOS:")
    for i, orf in enumerate(orfs[:5]):
        proteina_orf = adn_a_aminoacidos(orf['secuencia'], 0)
        
        print(f"\n{i+1}. ORF en posición {orf['inicio']}-{orf['fin']}")
        print(f"   Marco de lectura: {orf['frame']}")
        print(f"   Longitud: {orf['longitud']} nucleótidos ({orf['longitud']//3} aminoácidos)")
        print(f"   Proteína: {proteina_orf[:30]}...")
        print(f"   ADN: {orf['secuencia'][:60]}...")
else:
    print("⚠️  No se encontraron ORFs largos")

🧬 CONVERSIÓN COMPLETA DE SARS-CoV-2 A AMINOÁCIDOS

--- MARCO DE LECTURA 0 ---
🧬 Convirtiendo ADN a aminoácidos (marco de lectura 0)...
⚠️  Codón de parada encontrado en posición 8
✅ Conversión completada: 9 aminoácidos
Longitud de la proteína: 9 aminoácidos
Primeros 50 aminoácidos: IKGLYLPR*
Aminoácidos diferentes: 7 tipos
Codones de parada encontrados: 1
Codones no reconocidos: 0

--- MARCO DE LECTURA 1 ---
🧬 Convirtiendo ADN a aminoácidos (marco de lectura 1)...
⚠️  Codón de parada encontrado en posición 44
✅ Conversión completada: 45 aminoácidos
Longitud de la proteína: 45 aminoácidos
Primeros 50 aminoácidos: LKVYTFPGNKPTNFRSLVDLFSKRTLKSVWLSLGCMLSALTQYN*
Aminoácidos diferentes: 17 tipos
Codones de parada encontrados: 1
Codones no reconocidos: 0

--- MARCO DE LECTURA 2 ---
🧬 Convirtiendo ADN a aminoácidos (marco de lectura 2)...
⚠️  Codón de parada encontrado en posición 0
✅ Conversión completada: 1 aminoácidos
Longitud de la proteína: 1 aminoácidos
Primeros 50 aminoácidos: *
Aminoác

In [6]:
# Análisis avanzado: buscar motivos conservados usando el suffix trie
print("=" * 60)
print("🔬 ANÁLISIS DE MOTIVOS CONSERVADOS")
print("=" * 60)

def buscar_motivos_repetidos(trie, secuencia, longitud_minima=6, repeticiones_minimas=3):
    """
    Busca motivos que aparecen múltiples veces en la secuencia
    """
    print(f"Buscando motivos de al menos {longitud_minima} nucleótidos que aparecen {repeticiones_minimas}+ veces...")
    
    motivos_encontrados = []
    
    # Probar diferentes longitudes de motivos
    for longitud in range(longitud_minima, min(20, len(secuencia)//10)):
        print(f"  Analizando motivos de longitud {longitud}...")
        
        # Probar cada posición como inicio de motivo
        for inicio in range(len(secuencia) - longitud + 1):
            motivo = secuencia[inicio:inicio + longitud]
            
            # Buscar cuántas veces aparece este motivo
            posiciones = trie.buscar_patron(motivo)
            
            if len(posiciones) >= repeticiones_minimas:
                motivos_encontrados.append({
                    'motivo': motivo,
                    'longitud': longitud,
                    'repeticiones': len(posiciones),
                    'posiciones': posiciones[:10]  # Solo las primeras 10
                })
    
    # Eliminar duplicados y ordenar por repeticiones
    motivos_unicos = []
    motivos_vistos = set()
    
    for motivo_info in motivos_encontrados:
        motivo = motivo_info['motivo']
        if motivo not in motivos_vistos:
            motivos_vistos.add(motivo)
            motivos_unicos.append(motivo_info)
    
    # Ordenar por número de repeticiones
    motivos_unicos.sort(key=lambda x: x['repeticiones'], reverse=True)
    
    return motivos_unicos

# Buscar motivos conservados en la muestra
motivos = buscar_motivos_repetidos(trie_sars, muestra_secuencia)

if motivos:
    print(f"\n🎯 TOP 10 MOTIVOS MÁS CONSERVADOS:")
    for i, motivo in enumerate(motivos[:10]):
        print(f"\n{i+1}. Motivo: '{motivo['motivo']}'")
        print(f"   Longitud: {motivo['longitud']} nucleótidos")
        print(f"   Repeticiones: {motivo['repeticiones']}")
        print(f"   Posiciones: {motivo['posiciones']}")
        
        # Convertir a aminoácidos si es múltiplo de 3
        if motivo['longitud'] % 3 == 0:
            aa = adn_a_aminoacidos(motivo['motivo'], 0)
            print(f"   Aminoácidos: {aa}")
else:
    print("⚠️  No se encontraron motivos conservados significativos")

print(f"\n✅ Análisis completado!")

🔬 ANÁLISIS DE MOTIVOS CONSERVADOS
Buscando motivos de al menos 6 nucleótidos que aparecen 3+ veces...
  Analizando motivos de longitud 6...
  Analizando motivos de longitud 7...
  Analizando motivos de longitud 8...
  Analizando motivos de longitud 9...
  Analizando motivos de longitud 10...
  Analizando motivos de longitud 11...
  Analizando motivos de longitud 12...
  Analizando motivos de longitud 13...
  Analizando motivos de longitud 14...
  Analizando motivos de longitud 15...
  Analizando motivos de longitud 16...
  Analizando motivos de longitud 17...
  Analizando motivos de longitud 18...
  Analizando motivos de longitud 19...

🎯 TOP 10 MOTIVOS MÁS CONSERVADOS:

1. Motivo: 'GTGGCT'
   Longitud: 6 nucleótidos
   Repeticiones: 4
   Posiciones: [92, 355, 421, 615]
🧬 Convirtiendo ADN a aminoácidos (marco de lectura 0)...
✅ Conversión completada: 2 aminoácidos
   Aminoácidos: VA

2. Motivo: 'TTAAAG'
   Longitud: 6 nucleótidos
   Repeticiones: 3
   Posiciones: [6, 406, 871]
🧬 Convir

# Resumen del Análisis

## 🧬 Suffix Trie para SARS-CoV-2

Este notebook demostró cómo usar un **suffix trie simple** para:

### ✅ Funcionalidades Implementadas:
1. **Construcción de suffix trie** - Para análisis de secuencias de ADN
2. **Búsqueda de patrones** - Codones importantes, motivos conservados
3. **Conversión ADN→Aminoácidos** - Usando tabla genética estándar
4. **Detección de ORFs** - Open Reading Frames en múltiples marcos
5. **Análisis de motivos** - Secuencias repetidas y conservadas

### 📊 Análisis Realizado:
- **Composición nucleótida** del genoma SARS-CoV-2
- **Contenido GC** y estadísticas básicas
- **Traducción a proteínas** en 3 marcos de lectura
- **Identificación de ORFs** potenciales
- **Búsqueda de motivos** conservados

### 🎯 Aplicaciones:
- Análisis de genomas virales
- Búsqueda de genes y regiones codificantes
- Identificación de sitios de regulación
- Comparación de secuencias
- Diseño de primers y sondas

### ⚡ Complejidad:
- **Construcción del trie**: O(n²)
- **Búsqueda de patrones**: O(m) donde m = longitud del patrón
- **Conversión a aminoácidos**: O(n/3)